In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

# MES AÑO DE CONCILIACIÓN

In [2]:
month_conc = 'March'
year_conc = '2023'

# RUTAS DE ACCESO

### SWAP DEL DIA 

In [3]:
allfiles_disp_trad = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\2023\03. MARZO\ARCHIVO DISPERSIONES\Dispersión*.xlsx', recursive=False)
allfiles_disp_trad

['C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\03. MARZO\\ARCHIVO DISPERSIONES\\Dispersión-20230301.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\03. MARZO\\ARCHIVO DISPERSIONES\\Dispersión-20230302.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\03. MARZO\\ARCHIVO DISPERSIONES\\Dispersión-20230303.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\03. MARZO\\ARCHIVO DISPERSIONES\\Dispersión-20230304.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\03. MARZO\\ARCHIVO DISPERSIONES\\Dispersión-20230305.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\03. MARZO\\ARCHIVO DISPERSIONES\\Dispersión-20230306.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\03. MARZO\\ARCHIVO DISPERSIONES\\Dispersión-20230307.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2023\\03. MARZO\\ARCHIVO DISPERSIONES\\Dispersión-20230308.xlsx',
 'C:\\Users\\Cri

### FAST PAYMENTS

In [4]:
allfiles_fp = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2023\03. MARZO\ARCHIVO DISPERSIONES FP\Dispersión-FP*.xlsx',recursive=False)
allfiles_fp

['C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2023\\03. MARZO\\ARCHIVO DISPERSIONES FP\\Dispersión-FP 2023-03-01.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2023\\03. MARZO\\ARCHIVO DISPERSIONES FP\\Dispersión-FP 2023-03-02.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2023\\03. MARZO\\ARCHIVO DISPERSIONES FP\\Dispersión-FP 2023-03-03.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2023\\03. MARZO\\ARCHIVO DISPERSIONES FP\\Dispersión-FP 2023-03-04.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2023\\03. MARZO\\ARCHIVO DISPERSIONES FP\\Dispersión-FP 2023-03-05.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2023\\03. MARZO\\ARCHIVO DISPERSIONES FP\\Dispersión-FP 2023-03-06.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2023\\03. MARZO\\ARCHIVO DISPERSIONES FP\\Dispersión-FP 2023-03-07.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2023\\03. MARZO\\A

# SWAP DEL DIA

In [ ]:
swap_dia_total = pd.concat([pd.read_excel(f,sheet_name = 'Swap del día')
                            .assign(file_name=os.path.basename(f)) for f in allfiles_disp_trad])
swap_dia_total['Date'] = swap_dia_total["file_name"].str[-13:-5]
swap_dia_total['Date']= pd.to_datetime(swap_dia_total['Date'], format='%Y%m%d').dt.date
swap_dia_total['Date']= pd.to_datetime(swap_dia_total['Date'])
swap_dia_total.sort_values(by=['Date'],ascending=True,inplace=True)
swap_dia_total['report_date_local']= pd.to_datetime(swap_dia_total['report_date_local'])
swap_dia_total['month_conc'] = month_conc
swap_dia_total['year_conc'] = year_conc
swap_dia_total['month_report'] = swap_dia_total['report_date_local'].dt.strftime('%B')
swap_dia_total['year_report'] = swap_dia_total['report_date_local'].dt.year.astype(str)
swap_dia_total.head()

In [ ]:
swap_dia_total.shape

In [ ]:
swap_dia_total.dtypes

# FAST PAYMENTS

In [ ]:
fp_dia_total = pd.concat([pd.read_excel(f,sheet_name='FAST PAYMENTS')
                          .assign(file_name=os.path.basename(f)) for f in allfiles_fp])
fp_dia_total['Date'] = fp_dia_total["file_name"].str[-15:-5]
fp_dia_total['Date']= pd.to_datetime(fp_dia_total['Date'])
fp_dia_total['report_date_local']= pd.to_datetime(fp_dia_total['report_date_local'])
fp_dia_total['month_conc'] = month_conc
fp_dia_total['year_conc'] = year_conc
fp_dia_total['month_report'] = fp_dia_total['report_date_local'].dt.strftime('%B')
fp_dia_total['year_report'] = fp_dia_total['report_date_local'].dt.year.astype(str)
fp_dia_total.head()

In [ ]:
fp_dia_total.shape

In [ ]:
fp_dia_total.dtypes

# ALL DISPERSIONES

In [ ]:
all_disp = pd.concat([swap_dia_total,fp_dia_total])
all_disp.loc[all_disp['type'].str.contains('LEDGER'), 'type'] = 'LEDGER'
all_disp.head()

In [ ]:
all_disp.shape

In [ ]:
all_disp['type'].unique()

# TRADICIONAL

In [ ]:
all_disp_trad = all_disp[(all_disp['type'] == 'DAILY')]
all_disp_trad.head()

In [ ]:
all_disp_trad.shape

In [ ]:
pivot_total_trad = pd.pivot_table(all_disp_trad,index=['type','Date'],values=['total_due'],
                                  aggfunc = ['sum', len],
                                  margins=True)
pivot_total_trad.reset_index(drop=False, inplace=True)
pivot_total_trad.head()

In [ ]:
pivot_parcial_trad = pd.pivot_table(all_disp_trad,index=['type','tipo','Date'],
                               columns=['status'],values=['total_due'],aggfunc = ['sum', len],margins=True)
pivot_parcial_trad.reset_index(drop=False, inplace=True)
pivot_parcial_trad.head()

## CATEGORIZACIÓN DE REPROGRAMACIONES

In [ ]:
swap_dia_total_repro = all_disp_trad[(all_disp_trad['tipo'] == 'Reprogramación')]
swap_dia_total_repro.head()

In [ ]:
swap_dia_total_repro.shape

In [ ]:
swap_dia_total_repro.to_excel('Reprogramaciones_daily.xlsx',index=False)

In [ ]:
swap_dia_total_repro.loc[(swap_dia_total_repro['tipo'] == 'Reprogramación')
                   & (swap_dia_total_repro['month_report'] == month_conc) 
                   & (swap_dia_total_repro['year_report'] == year_conc),'tipo_reprogramacion'] = 'repro_mismo_mes'
swap_dia_total_repro['tipo_reprogramacion'].fillna('repro_otro_mes',inplace=True)
swap_dia_total_repro.head()

In [ ]:
pivot_tipo_repro = pd.pivot_table(swap_dia_total_repro,index=['type','Date'],
                                  columns=['tipo_reprogramacion'],
                                  values=['total_due']
                                  ,aggfunc = ['sum', len],margins=True)
pivot_tipo_repro.reset_index(drop=False, inplace=True)
pivot_tipo_repro.head()

# FAST PAYMENTS

In [ ]:
all_disp_fp = all_disp[(all_disp['type'] == 'INSTANT')]
all_disp_fp.head()

In [ ]:
all_disp_fp.shape

In [ ]:
pivot_total_fp = pd.pivot_table(all_disp_fp,index=['type','Date'],values=['total_due'],
                                  aggfunc = ['sum', len],
                                  margins=True)
pivot_total_fp.reset_index(drop=False, inplace=True)
pivot_total_fp.head()

In [ ]:
pivot_parcial_fp = pd.pivot_table(all_disp_fp,index=['type','tipo','Date'],
                               columns=['status'],values=['total_due'],aggfunc = ['sum', len],margins=True)
pivot_parcial_fp.reset_index(drop=False, inplace=True)
pivot_parcial_fp.head()

## CATEGORIZACIÓN DE REPROGRAMACIONES - FAST PAYMENTS

In [ ]:
fp_dia_total_repro = all_disp_fp[(all_disp_fp['tipo'] == 'Reprogramación')]
fp_dia_total_repro.head()

In [ ]:
fp_dia_total_repro.shape

In [ ]:
fp_dia_total_repro.to_excel('Reprogramaciones_instant.xlsx',index=False)

In [ ]:
fp_dia_total_repro.loc[(fp_dia_total_repro['tipo'] == 'Reprogramación')
                   & (fp_dia_total_repro['month_report'] == month_conc) 
                   & (fp_dia_total_repro['year_report'] == year_conc),'tipo_reprogramacion'] = 'repro_mismo_mes'
fp_dia_total_repro['tipo_reprogramacion'].fillna('repro_otro_mes',inplace=True)
fp_dia_total_repro.head()

In [ ]:
pivot_tipo_repro_fp = pd.pivot_table(fp_dia_total_repro,index=['type','Date'],
                                  columns=['tipo_reprogramacion'],
                                  values=['total_due']
                                  ,aggfunc = ['sum', len],margins=True)
pivot_tipo_repro_fp.fillna(0,inplace=True)
pivot_tipo_repro_fp.reset_index(drop=False, inplace=True)
pivot_tipo_repro_fp.head()

# LEDGER

In [ ]:
all_disp_ledger = all_disp[(all_disp['type'] == 'LEDGER')]
all_disp_ledger.head()

In [ ]:
all_disp_ledger.shape

In [ ]:
pivot_total_ledger = pd.pivot_table(all_disp_ledger,index=['type','Date'],values=['total_due'],
                                  aggfunc = ['sum', len],
                                  margins=True)
pivot_total_ledger.reset_index(drop=False, inplace=True)
pivot_total_ledger.head()

In [ ]:
pivot_total_ledger.shape

In [ ]:
pivot_parcial_ledger = pd.pivot_table(all_disp_ledger,index=['type','tipo','Date'],
                               columns=['status'],values=['total_due'],aggfunc = ['sum', len],margins=True)
pivot_parcial_ledger.reset_index(drop=False, inplace=True)
pivot_parcial_ledger.head()

## CATEGORIZACION DE REPROGRAMACIONES - LEDGER 

In [ ]:
ledger_dia_total_repro = all_disp_ledger[(all_disp_ledger['tipo'] == 'Reprogramación')]
ledger_dia_total_repro.head()

In [ ]:
ledger_dia_total_repro.shape

In [ ]:
ledger_dia_total_repro.to_excel('Reprogramaciones_ledger.xlsx',index=False)

In [ ]:
ledger_dia_total_repro.loc[(ledger_dia_total_repro['tipo'] == 'Reprogramación')
                   & (ledger_dia_total_repro['month_report'] == month_conc) 
                   & (ledger_dia_total_repro['year_report'] == year_conc),'tipo_reprogramacion'] = 'repro_mismo_mes'
ledger_dia_total_repro['tipo_reprogramacion'].fillna('repro_otro_mes',inplace=True)
ledger_dia_total_repro.head()

In [ ]:
pivot_tipo_repro_ledger = pd.pivot_table(ledger_dia_total_repro,index=['type','Date'],
                                  columns=['tipo_reprogramacion'],
                                  values=['total_due']
                                  ,aggfunc = ['sum', len],margins=True)
pivot_tipo_repro_ledger.fillna(0,inplace=True)
pivot_tipo_repro_ledger.reset_index(drop=False, inplace=True)
pivot_tipo_repro_ledger.head()

# EXCEL FINAL

In [ ]:
writer = pd.ExcelWriter('FASE IV. REPROGRAMACIONES MARZO 2023.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
pivot_total_trad.to_excel(writer, sheet_name='TRADICIONAL TOTAL')
pivot_parcial_trad.to_excel(writer, sheet_name='TRADICIONAL PARCIAL')
pivot_total_fp.to_excel(writer, sheet_name='FP TOTAL')
pivot_parcial_fp.to_excel(writer, sheet_name='FP PARCIAL')
pivot_total_ledger.to_excel(writer, sheet_name='LEDGER TOTAL')
pivot_parcial_ledger.to_excel(writer, sheet_name='LEDGER PARCIAL')
pivot_tipo_repro.to_excel(writer, sheet_name='CATEGORIZACION REPRO TRAD')
pivot_tipo_repro_fp.to_excel(writer, sheet_name='CATEGORIZACION REPRO FP')
pivot_tipo_repro_ledger.to_excel(writer, sheet_name='CATEGORIZACION REPRO LEDGER')
writer.close()